# Projeto 1 - Análise de Dados da PRF #

## Objetivo Inicial: ##
1. Carregar os dataframes da PRF referente aos dados de infrações da PRF do ano de 2021.

Os dados estão armazenados em um link do Google Drive, em um arquivo comprimido (*zipado*):

https://drive.google.com/file/d/1kgHHk0lurTuu4ZEn4Vhfo5W0ejOvDnZJ/view

Para fazer o download por meio do requests em Python, é preciso obter o link direto do download. De que forma isso é feito? Acessamos o link acima e verificamos o código fonte da página. O botão de download deve conter um link longo para o download direto do arquivo.

In [1]:
import locale # Para rodar alguns dados em português
import numpy as np
import pandas as pd # pandas e numpy para manipular os dataframes
import zipfile as zf # zipfile para manipular o arquivo zip salvo no Google Drive
import requests # Biblioteca para enviar requisições a endereços da internet e obter dados

url = "https://drive.google.com/u/0/uc?id=1kgHHk0lurTuu4ZEn4Vhfo5W0ejOvDnZJ&export=download&confirm=t&uuid=8484b178-eb44-4af5-a94c-a9496d3263ab&at=ALgDtswaioeBbCKyEKoLNm_tvmXo:1678822412025"
request = requests.get(url)
open("prf2021.zip", "wb").write(request.content)


121266241

Com o download feito, é simples conferir o arquivo .zip com o ZipFile.

In [2]:
z = zf.ZipFile("prf2021.zip", mode="r")
print(z.namelist())

['infraçoes2021_01.csv', 'infraçoes2021_02.csv', 'infraçoes2021_03.csv', 'infraçoes2021_04.csv', 'infraçoes2021_05.csv', 'infraçoes2021_06.csv', 'infraçoes2021_07.csv', 'infraçoes2021_08.csv', 'infraçoes2021_09.csv', 'infraçoes2021_10.csv', 'infraçoes2021_11.csv', 'infraçoes2021_12.csv']


Em seguida, é preciso transformar todos esses dados separados em um único *dataframe*.
Para isso, primeiro é necessário extrair os arquivos.

In [3]:
z.extractall()

Com a extração dos arquivos devidamente realizada, o próximo passo é criar o dataframe em si.

In [4]:
df = pd.DataFrame()
df = pd.concat([pd.read_csv(arquivo, sep=";", dtype={"Medição Considerada" : str, "Excesso Verificado": str}, 
                                encoding="cp1252") for arquivo in z.namelist()])

No trecho de código acima foi necessário fazer algum uso de tentativa e erro até encontrar os parâmetros ideais.
Primeiro, ao abrir o arquivo foi possível averiguar que ele é separado por ponto e vírgula em vez da tradicional vírgula dos csvs, por isso o uso de *sep=";"*. Depois disso, ainda houve um problema: um aviso de tipos de dados mistos em duas colunas diferentes.

*Observação: Aparentemente o problema com o tipo misto de arquivo só ocorre ao utilizar o notebook no Jupyter. Como este notebook foi finalizado no Google Colab, mantive o trecho apenas por questão de compatibilidade.*

Verificando os arquivos foi possível constatar seu formato de texto, conforme exemplo abaixo:

In [5]:
with open('infraçoes2021_01.csv') as f:
   print(f)

<_io.TextIOWrapper name='infraçoes2021_01.csv' mode='r' encoding='UTF-8'>


Por último, diversos *DType Warning* foram emitidos a respeito de duas colunas com tipos mistos de arquivo. Esses avisos não geram grandes problemas e eu poderia inclusive prosseguir com eles, mas optei por verificar as duas colunas com arquivos mistos e declará-los na criação do dataframe, eliminando os avisos desnecessários.

Resolvido os problemas iniciais, agora é possível conferir o resultado final do dataframe:

In [6]:
df.head()

,Número do Auto,Data da Infração (DD/MM/AAAA),Indicador de Abordagem,Assinatura do Auto,Indicador Veiculo Estrangeiro,Sentido Trafego,UF Infração,BR Infração,Km Infração,Município,...,Enquadramento da Infração,Início Vigência da Infração,Fim Vigência Infração,Medição Infração,Descrição Especie Veículo,Descrição Marca Veículo,Hora Infração,Medição Considerada,Excesso Verificado,Qtd Infrações
0,106198fa7cfe8e9,2021-01-08,S,N,N,C,ES,101,242,SERRA,...,209,2016-11-01,NaN,Nenhuma,TRAÇÃO,VOLVO,1,0,0,1
1,12963b279370873,2021-01-08,S,N,N,D,ES,101,242,SERRA,...,209,2016-11-01,NaN,Nenhuma,TRAÇÃO,VOLVO,16,0,0,1
2,6b7a329810761b4,2021-01-10,S,N,N,C,ES,101,242,SERRA,...,209,2016-11-01,NaN,Nenhuma,TRAÇÃO,OUTRA,7,0,0,1
3,a866e8850da632b,2021-01-11,S,N,N,C,ES,101,242,SERRA,...,209,2016-11-01,NaN,Nenhuma,MISTO,OUTRA,19,0,0,1
4,77bf454bcb08f99,2021-01-12,S,N,N,D,ES,101,242,SERRA,...,209,2016-11-01,NaN,Nenhuma,CARGA,MERCEDES BENZ,11,0,0,1


Enquanto o *dataframe.head()* fornece uma boa visão geral da estrutura, o *dataframe.info()* pode ajudar a conhecer melhor as variáveis.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5241299 entries, 0 to 414770
Data columns (total 22 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Número do Auto                 object 
 1   Data da Infração (DD/MM/AAAA)  object 
 2   Indicador de Abordagem         object 
 3   Assinatura do Auto             object 
 4   Indicador Veiculo Estrangeiro  object 
 5   Sentido Trafego                object 
 6   UF Infração                    object 
 7   BR Infração                    int64  
 8   Km Infração                    int64  
 9   Município                      object 
 10  Código da Infração             int64  
 11  Descrição Abreviada Infração   object 
 12  Enquadramento da Infração      object 
 13  Início Vigência da Infração    object 
 14  Fim Vigência Infração          float64
 15  Medição Infração               object 
 16  Descrição Especie Veículo      object 
 17  Descrição Marca Veículo        object 
 18  Hor

Primeiro objetivo concluído.

## Segundo objetivo ##

responder às seguintes perguntas:
1. Quantas infrações ocorreram em cada mês?
2. Quantas variáveis cada conjunto de dados registra?
3. Quais as cinco cidades brasileiras onde mais ocorreram infrações em rodovias federais?

Para analisarmos as infrações por mês, o melhor caminho é converter as datas do dataframe para o formato DateTime e, em seguida, adicioná-las como o índice. Para isso, o seguinte código é muito útil:

In [8]:
df_convert = df
df_convert["Data"] = pd.to_datetime(df_convert["Data da Infração (DD/MM/AAAA)"])
df_convert.set_index("Data", inplace=True)
df_convert = df_convert.drop("Data da Infração (DD/MM/AAAA)", axis="columns") # Para deletar a coluna antiga.
df_convert.head()

,Número do Auto,Indicador de Abordagem,Assinatura do Auto,Indicador Veiculo Estrangeiro,Sentido Trafego,UF Infração,BR Infração,Km Infração,Município,Código da Infração,...,Enquadramento da Infração,Início Vigência da Infração,Fim Vigência Infração,Medição Infração,Descrição Especie Veículo,Descrição Marca Veículo,Hora Infração,Medição Considerada,Excesso Verificado,Qtd Infrações
Data,,,,,,,,,,,,,,,,,,,,,
2021-01-08,106198fa7cfe8e9,S,N,N,C,ES,101,242,SERRA,60683,...,209,2016-11-01,NaN,Nenhuma,TRAÇÃO,VOLVO,1,0,0,1
2021-01-08,12963b279370873,S,N,N,D,ES,101,242,SERRA,60683,...,209,2016-11-01,NaN,Nenhuma,TRAÇÃO,VOLVO,16,0,0,1
2021-01-10,6b7a329810761b4,S,N,N,C,ES,101,242,SERRA,60683,...,209,2016-11-01,NaN,Nenhuma,TRAÇÃO,OUTRA,7,0,0,1
2021-01-11,a866e8850da632b,S,N,N,C,ES,101,242,SERRA,60683,...,209,2016-11-01,NaN,Nenhuma,MISTO,OUTRA,19,0,0,1
2021-01-12,77bf454bcb08f99,S,N,N,D,ES,101,242,SERRA,60683,...,209,2016-11-01,NaN,Nenhuma,CARGA,MERCEDES BENZ,11,0,0,1


In [9]:
df_infmensal = df_convert.groupby(df.index.month)["Qtd Infrações"].sum()
df_infmensal = pd.DataFrame(df_infmensal)
df_infmensal

,Qtd Infrações
Data,
1,442458
2,441593
3,380688
4,374764
5,412057
6,456576
7,470598
8,496155
9,503323


Quanto a segunda pergunta (o número de variáveis), ela já foi respondida. Basta usarmos *df.info()* para obter todas as variáveis disponíveis, como segue:

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5241299 entries, 2021-01-08 to 2021-12-30
Data columns (total 22 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Número do Auto                 object 
 1   Data da Infração (DD/MM/AAAA)  object 
 2   Indicador de Abordagem         object 
 3   Assinatura do Auto             object 
 4   Indicador Veiculo Estrangeiro  object 
 5   Sentido Trafego                object 
 6   UF Infração                    object 
 7   BR Infração                    int64  
 8   Km Infração                    int64  
 9   Município                      object 
 10  Código da Infração             int64  
 11  Descrição Abreviada Infração   object 
 12  Enquadramento da Infração      object 
 13  Início Vigência da Infração    object 
 14  Fim Vigência Infração          float64
 15  Medição Infração               object 
 16  Descrição Especie Veículo      object 
 17  Descrição Marca Veículo        

Em seguida, criaremos um código para exibir as cinco cidades com o maior número de infrações:

In [11]:
df_ranking = df.groupby(df.Município)["Qtd Infrações"].sum().sort_values(ascending=False)
df_ranking.head()

Município
RIO DE JANEIRO    194011
NITEROI           173277
MAIRIPORA         136054
BELO HORIZONTE    124818
ITABORAI          112712
Name: Qtd Infrações, dtype: int64

In [12]:
# A título de curiosidade, decidi conferir quantas infrações foram contabilizadas no município de Campina Grande, de João Pessoa, São João de Meriti e Macaé.

df_cg = df[df["Município"] == "CAMPINA GRANDE"]["Qtd Infrações"].sum()
df_jp = df[df["Município"] == "JOAO PESSOA"]["Qtd Infrações"].sum()
df_sjm = df[df["Município"] == "SAO JOAO DE MERITI"]["Qtd Infrações"].sum()
df_mc = df[df["Município"] == "MACAE"]["Qtd Infrações"].sum()
print(df_cg, df_jp, df_sjm, df_mc, sep="\n")

9195
8665
44688
797


Segundo objetivo concluído.

## Terceiro Objetivo ##
Criar um novo *dataframe* em formato CSV para cada uma das seguintes questões:

1. Ranking de Infrações por Estado
2. Infrações por dia da semana do ano inteiro

Começamos criando o primeiro dataframe:

In [13]:
df_ranking_estado = df.groupby("UF Infração")["Qtd Infrações"].sum()
df_ranking_estado = df_ranking_estado.sort_values(ascending=False)
df_ranking_estado = pd.DataFrame(df_ranking_estado)
df_ranking_estado

,Qtd Infrações
UF Infração,
RJ,925055
SP,700587
MG,685155
BA,462102
MS,361411
PR,258554
SC,235609
GO,209939
MT,185499


Antes de criar o arquivo, podemos nos adiantar e criar o segundo dataframe:

In [15]:
locale.setlocale(locale.LC_ALL, locale='pt_BR.utf8')
#sudo dpkg-reconfigure locales 
# Isso aqui foi feito para instalar o arquivo de tradução correto. Se quiser fazer da mesma forma,
# basta "descomentar" o código acima (remover o "#" antes da exclamação),
# selecionar o valor 380 na caixa de seleção que aparecerá e definir pt_BR como padrão. 
# Pode ser necessário reiniciar o Ambiente de Execução. 
# Para ler os dados em inglês, basta alterar "locale" da linha abaixo para None.
# Em resumo: se der algum erro e quiser ler os valores em inglês, alterar "locale" abaixo para None.
# Esse problema é específico do Google Colab, então infelizmente não há o que fazer.
df_dias_semana = df_convert.groupby(df.index.day_name(locale="pt_BR.utf-8"))["Qtd Infrações"].sum().sort_values(ascending=False)
df_dias_semana = pd.DataFrame(df_dias_semana)
df_dias_semana

,Qtd Infrações
Data,
Sexta,818925
Domingo,760256
Sábado,743481
Quinta,735974
Quarta,732179
Terça,730309
Segunda,720175


Feito isso, agora é só salvar os dataframes.

In [16]:
df_ranking_estado.to_csv("df_ranking_estado.csv", index=True)
df_dias_semana.to_csv("df_dias_semana.csv", index=True)

Conferindo os arquivos:

In [17]:
conferir_1 = pd.read_csv("df_ranking_estado.csv")
conferir_2 = pd.read_csv("df_dias_semana.csv")
conferir_1

,UF Infração,Qtd Infrações
0,RJ,925055
1,SP,700587
2,MG,685155
3,BA,462102
4,MS,361411
5,PR,258554
6,SC,235609
7,GO,209939
8,MT,185499
9,ES,181184


In [18]:
conferir_2

,Data,Qtd Infrações
0,Sexta,818925
1,Domingo,760256
2,Sábado,743481
3,Quinta,735974
4,Quarta,732179
5,Terça,730309
6,Segunda,720175
